### Asr evaluation

- Будем тестировать несколько аср моделей на общих сетах + на сете с голосовыми командами на русском языке для редактирования
- Тестить собираемся:
    - Whisper
    - GigaAM-v3
    - T-one
    - Vosk
    - ...

In [1]:
import librosa
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import AutoModel

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# Whisper

whisper_model_name = "openai/whisper-small"
whisper_processor = WhisperProcessor.from_pretrained(whisper_model_name)
whisper_model = WhisperForConditionalGeneration.from_pretrained(whisper_model_name).to(device)

In [4]:
def transcribe_whisper(audio_input, sr=None, language='ru'):
    WHISPER_SR = 16000
    
    if isinstance(audio_input, str):
        audio_array = librosa.load(audio_input, sr=WHISPER_SR, mono=True)[0]
        sr = WHISPER_SR
    else:
        audio_array = audio_input
        if sr is None:
            sr = WHISPER_SR
        elif sr != WHISPER_SR:
            audio_array = librosa.resample(audio_array, orig_sr=sr, target_sr=WHISPER_SR)
            sr = WHISPER_SR
    
    input_features = whisper_processor(audio_array, sampling_rate=sr, return_tensors="pt").input_features.to(device)
    
    generated_ids = whisper_model.generate(
        input_features,
        language=language,
        task="transcribe"
    )
    
    transcription = whisper_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return transcription.lower()

In [11]:
transcribe_whisper('/data/sallundina/operators_09_25/operators_09_25/Jukov/000000_015075_Jukov.wav')

' я понял.'

In [6]:
transcribe_whisper('/data/sallundina/tts_datasets_rus/Pik_speaker/original_audio/fee5af9e141f5db38ef48e4898c78c66.flac')

' время ожидания составит 10 минут.'

In [7]:
# GigaAM-v3

revision = "e2e_rnnt"
model = AutoModel.from_pretrained(
    "ai-sage/GigaAM-v3",
    revision=revision,
    trust_remote_code=True,
)

In [8]:
transcription = model.transcribe('/data/sallundina/operators_09_25/operators_09_25/Jukov/000000_015075_Jukov.wav')
print(transcription)

Я понял.


In [9]:
transcription = model.transcribe('/data/sallundina/tts_datasets_rus/Pik_speaker/original_audio/fee5af9e141f5db38ef48e4898c78c66.flac')
print(transcription)

Время ожидания составит 10 минут.


In [10]:
# T-one

t_one_model_name = "t-tech/T-one"